In [9]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
)

In [10]:
MIRA_PATH = "test/models/april_ensemble_demo/"
SAVE_PATH = "notebook/integration_demo/"

filename = "BIOMD0000000955_template_model.json"
filename = os.path.join(MIRA_PATH, filename)

## load_and_sample_petri_model

In [13]:
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
samples = load_and_sample_petri_model(
    filename, num_samples, timepoints=timepoints, add_uncertainty=True
)

# Save results
samples.to_csv(os.path.join(SAVE_PATH, "results_petri/sample_results.csv"))

## load_and_calibrate_and_sample_petri_model

In [14]:
# List of tuples of the form `(time, observation_dict)` where each `observation_dict` is of the form `{observable_name: observable_value}`.
# Once we get a data file format we're very happy to handle the csv -> this format processing.
# Note: As you can see here, not every variable must be observed at every timepoint.
data = [
    (1.1, {"Susceptible": 0.9, "Infected": 0.1}),
    (2.2, {"Susceptible": 0.8}),
]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    filename,
    data,
    num_samples,
    timepoints=timepoints,
    add_uncertainty=True,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(SAVE_PATH, "results_petri/calibrated_sample_results.csv")
)

iteration 0: loss = 63.796528339385986
iteration 25: loss = 58.426074743270874
iteration 50: loss = 49.85667538642883
iteration 75: loss = 44.239312410354614
iteration 100: loss = 35.20773935317993
iteration 125: loss = 36.89239001274109
iteration 150: loss = 31.214526176452637
iteration 175: loss = 31.498044967651367
iteration 200: loss = 30.04111909866333
iteration 225: loss = 30.397873878479004
iteration 250: loss = 30.3555109500885
iteration 275: loss = 31.16167974472046
iteration 300: loss = 30.726374626159668
iteration 325: loss = 29.893669843673706
iteration 350: loss = 30.01099967956543
iteration 375: loss = 29.83716106414795
iteration 400: loss = 31.648341178894043
iteration 425: loss = 30.367257118225098
iteration 450: loss = 31.26539659500122
iteration 475: loss = 29.29333209991455
iteration 500: loss = 30.23838233947754
iteration 525: loss = 29.057374715805054
iteration 550: loss = 30.36006498336792
iteration 575: loss = 30.363718271255493
iteration 600: loss = 28.328775882